In [ ]:
!pip install torchmetrics
!pip install pytorch_lightning

In [1]:
import collections
import copy
import os

import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from absl import app, flags
from skimage import io
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from tqdm import tqdm

FLAGS = flags.FLAGS

flags.DEFINE_enum('task_type', 'training', ['training', 'analysis'],
                  'Specifies the task type.')

# Hyperparameters for Part I
flags.DEFINE_float('learning_rate', 1e-3, 'Learning rate.')
flags.DEFINE_float('weight_decay', 0, 'Weight decay (L2 regularization).')
flags.DEFINE_integer('batch_size', 128, 'Number of examples per batch.')
flags.DEFINE_integer('epochs', 100, 'Number of epochs for training.')
flags.DEFINE_string('experiment_name', 'exp', 'Defines experiment name.')
flags.DEFINE_enum('label_type', 'domain', ['domain', 'category'],
                  'Specifies prediction task.')

# Hyperparemeters for Part III
flags.DEFINE_string('model_checkpoint', '',
                    'Specifies the checkpont for analyzing.')

LABEL_SIZE = {'domain': 4, 'category': 7}



In [2]:
import torchmetrics
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch import nn, optim
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

# **1. Implement AlexNet**

**PACSDataset**

In [3]:
class PACSDataset(Dataset):

  def __init__(self,
               root_dir,
               label_type='domain',
               is_training=False,
               transform=None):
    self.root_dir = os.path.join(root_dir, 'train' if is_training else 'val')
    self.label_type = label_type
    self.is_training = is_training
    if transform:
      self.transform = transform
    else:
      self.transform = transforms.Compose([
          transforms.ToTensor(),
          transforms.Normalize(mean=[0.7659, 0.7463, 0.7173],
                               std=[0.3089, 0.3181, 0.3470]),
      ])

    self.dataset, self.label_list = self.initialize_dataset()
    self.label_to_id = {x: i for i, x in enumerate(self.label_list)}
    self.id_to_label = {i: x for i, x in enumerate(self.label_list)}

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    image, label = self.dataset[idx]
    label_id = self.label_to_id[label]
    image = self.transform(image)
    return image, label_id

  def initialize_dataset(self):
    assert os.path.isdir(self.root_dir), \
        '`root_dir` is not found at %s' % self.root_dir

    dataset = []
    domain_set = set()
    category_set = set()
    cnt = 0

    for root, dirs, files in os.walk(self.root_dir, topdown=True):
      if files:
        _, domain, category = root.rsplit('/', maxsplit=2)
        domain_set.add(domain)
        category_set.add(category)
        pbar = tqdm(files)
        for name in pbar:
          pbar.set_description('Processing Folder: domain=%s, category=%s' %
                               (domain, category))
          img_array = io.imread(os.path.join(root, name))
          dataset.append((img_array, domain, category))

    images, domains, categories = zip(*dataset)

    if self.label_type == 'domain':
      labels = sorted(domain_set)
      dataset = list(zip(images, domains))
    elif self.label_type == 'category':
      labels = sorted(category_set)
      dataset = list(zip(images, categories))
    else:
      raise ValueError(
          'Unknown `label_type`: Expecting `domain` or `category`.')

    return dataset, labels


**AlexNet**

In [4]:
class AlexNet(nn.Module):

  def __init__(self, configs):
    super().__init__()
    self.configs = configs
    super().__init__()
    act_func = nn.ReLU()
    self.loss = nn.CrossEntropyLoss()
    self.lr = float(1e-3)
    self.dropout = 0.2
    self.train_acc = torchmetrics.Accuracy()
    self.val_acc = torchmetrics.Accuracy()
    self.features = nn.Sequential(
        nn.Conv2d(3, 96, kernel_size=11, stride=4),
        act_func,
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.Conv2d(96, 256, kernel_size=5, padding=2),
        act_func,
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.Conv2d(256, 384, kernel_size=3, padding=1),
        act_func,
        nn.Conv2d(384, 384, kernel_size=3, padding=1),
        act_func,
        nn.Conv2d(384, 256, kernel_size=3, padding=1),
        act_func,
        nn.MaxPool2d(kernel_size=3, stride=2),
    )
    self.classifier = nn.Sequential(
        nn.Flatten(-1),
        nn.Dropout(self.dropout),
        nn.Linear(9216, 4096),
        act_func,
        nn.Dropout(self.dropout),
        nn.Linear(4096, 4096),
        act_func,
        nn.Linear(4096, 28),
    )

  def forward(self, X):
      X = self.features(X)
      batch_size, _, _, _ = X.size()
      X = X.view(batch_size, -1)
      X = self.classifier(X)
      return X


In [5]:
def model_training(model, root_dir, label_type, batch_size, max_epochs, learning_rate, weight_decay, experiment_name):
  train_dataset = PACSDataset(root_dir=root_dir,
                              label_type=label_type,
                              is_training=True)
  train_loader = DataLoader(train_dataset,
                            batch_size=batch_size,
                            shuffle=True,
                            num_workers=4)

  val_dataset = PACSDataset(root_dir=root_dir,
                            label_type=label_type,
                            is_training=False)
  val_loader = DataLoader(val_dataset,
                          batch_size=batch_size,
                          shuffle=False,
                          num_workers=4)

  best_model = None
  best_acc = 0.0

  experiment_name = 'experiments/{}/{}_lr_{}.wd_{}'.format(
      experiment_name, label_type, learning_rate,
      weight_decay)

  os.makedirs(experiment_name, exist_ok=True)
  writer = SummaryWriter(log_dir=experiment_name)

  ############################################################################
  """After implementing all required models, you can switch from here."""
  # model = AlexNet(configs).to(device)
  # model = AlexNetLargeKernel(configs).to(device)
  # model = AlexNetAvgPooling(configs).to(device)
  ############################################################################

  print('Model Architecture:\n%s' % model)

  criterion = nn.CrossEntropyLoss(reduction='mean')
  optimizer = torch.optim.Adam(model.parameters(),
                               lr=learning_rate,
                               weight_decay=weight_decay)

  try:
    for epoch in range(max_epochs):
      for phase in ('train', 'eval'):
        if phase == 'train':
          model.train()
          dataset = train_dataset
          data_loader = train_loader
        else:
          model.eval()
          dataset = val_dataset
          data_loader = val_loader

        running_loss = 0.0
        running_corrects = 0

        for step, (images, labels) in enumerate(data_loader):
          images = images.to(device)
          labels = labels.to(device)

          optimizer.zero_grad()

          with torch.set_grad_enabled(phase == 'train'):
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            if phase == 'train':
              loss.backward()
              optimizer.step()

              writer.add_scalar('Loss/{}'.format(phase), loss.item(),
                                epoch * len(data_loader) + step)

          running_loss += loss.item() * images.size(0)
          running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataset)
        epoch_acc = running_corrects.double() / len(dataset)
        writer.add_scalar('Epoch_Loss/{}'.format(phase), epoch_loss, epoch)
        writer.add_scalar('Epoch_Accuracy/{}'.format(phase), epoch_acc, epoch)
        print('[Epoch %d] %s accuracy: %.4f, loss: %.4f' %
              (epoch + 1, phase, epoch_acc, epoch_loss))

        if phase == 'eval':
          if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model = copy.deepcopy(model.state_dict())
            torch.save(best_model, os.path.join(experiment_name,
                                                'best_model.pt'))

  except KeyboardInterrupt:
    pass

  return model

**predicting domain**

In [ ]:
root_dir='/content/drive/MyDrive/DL/HW2/pacs_dataset'

LABEL_SIZE = {'domain': 4, 'category': 7}
label_type = 'domain'
epochs = 50
lr=1e-3
weight_decay=0
batch_size=128
experiment_name= 'exp'

configs = {'num_classes': LABEL_SIZE[label_type]}

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

alexnet_domain = model_training(model= AlexNet(configs).to(device), 
                                root_dir=root_dir, 
                                label_type=label_type, 
                                batch_size=batch_size, 
                                max_epochs=epochs, 
                                learning_rate=lr, 
                                weight_decay=weight_decay,
                                experiment_name=experiment_name)

Processing Folder: domain=photo, category=dog: 100%|██████████| 169/169 [00:03<00:00, 48.40it/s]
Processing Folder: domain=photo, category=elephant: 100%|██████████| 181/181 [00:03<00:00, 56.98it/s]
Processing Folder: domain=photo, category=giraffe: 100%|██████████| 165/165 [00:03<00:00, 53.56it/s]
Processing Folder: domain=photo, category=person: 100%|██████████| 383/383 [00:06<00:00, 56.12it/s]
Processing Folder: domain=art_painting, category=giraffe: 100%|██████████| 254/254 [00:07<00:00, 34.77it/s]
Processing Folder: domain=art_painting, category=house: 100%|██████████| 262/262 [00:05<00:00, 51.97it/s]
Processing Folder: domain=art_painting, category=dog: 100%|██████████| 348/348 [00:06<00:00, 54.34it/s]
Processing Folder: domain=art_painting, category=person: 100%|██████████| 404/404 [00:07<00:00, 52.86it/s]
Processing Folder: domain=art_painting, category=horse: 100%|██████████| 179/179 [00:02<00:00, 60.29it/s]
Processing Folder: domain=art_painting, category=elephant: 100%|█████

Model Architecture:
AlexNet(
  (loss): CrossEntropyLoss()
  (train_acc): Accuracy()
  (val_acc): Accuracy()
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=-1, end_dim=-1)
    (1): Dropout(p=0.2, inplace=False)
    (2): Linear(in_features=9216, out_features=409

**predicting class**

In [ ]:
root_dir='/content/drive/MyDrive/DL/HW2/pacs_dataset'

LABEL_SIZE = {'domain': 4, 'category': 7}
label_type = 'category'
epochs = 50
lr=1e-3
weight_decay=0
batch_size=128
experiment_name= 'exp'

configs = {'num_classes': LABEL_SIZE[label_type]}

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

alexnet_category = model_training(model= AlexNet(configs).to(device), 
                                root_dir=root_dir, 
                                label_type=label_type, 
                                batch_size=batch_size, 
                                max_epochs=epochs, 
                                learning_rate=lr, 
                                weight_decay=weight_decay,
                                experiment_name=experiment_name)


# 2. Enhancing AlexNet

**AlexNetLargeKernel**

In [ ]:
class AlexNetLargeKernel(nn.Module):
  def __init__(self, configs):
    
        super().__init__()
        self.configs = configs
        act_func = nn.ReLU()
        self.loss = nn.CrossEntropyLoss()
        self.lr = float(1e-3)
        self.dropout = 0.2
        self.train_acc = torchmetrics.Accuracy()
        self.val_acc = torchmetrics.Accuracy()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=21, padding=1, stride=8),
            act_func,
            nn.Conv2d(96, 256, kernel_size=7, padding=2, stride=2),
            act_func,
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            act_func,
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            act_func,
            nn.Conv2d(384, 256, kernel_size=3, stride=2),
            act_func,
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(-1),
            nn.Dropout(self.dropout),
            nn.Linear(1024, 4096),
            act_func,
            nn.Dropout(self.dropout),
            nn.Linear(4096, 4096),
            act_func,
            nn.Linear(4096, 28),
        )

  def forward(self, X):
      X = self.features(X)
      batch_size, _, _, _ = X.size()
      X = X.view(batch_size, -1)
      X = self.classifier(X)
      return X


**AlexNetAvgPooling**

In [ ]:
class AlexNetAvgPooling(nn.Module):
  def __init__(self, configs):
    super().__init__()
    act_func = nn.ReLU()
    self.loss = nn.CrossEntropyLoss()
    self.lr = float(1e-3)
    self.dropout = 0.2
    self.train_acc = torchmetrics.Accuracy()
    self.val_acc = torchmetrics.Accuracy()
    self.features = nn.Sequential(
        nn.Conv2d(3, 96, kernel_size=11, stride=4),
        act_func,
        nn.AvgPool2d(3, 2),
        nn.Conv2d(96, 256, kernel_size=5, padding=2),
        act_func,
        nn.AvgPool2d(3, 2),
        nn.Conv2d(256, 384, kernel_size=3, padding=1),
        act_func,
        nn.Conv2d(384, 384, kernel_size=3, padding=1),
        act_func,
        nn.Conv2d(384, 256, kernel_size=3, stride=2),
        act_func,
        nn.MaxPool2d(kernel_size=3, stride=2),
    )
    self.classifier = nn.Sequential(
        nn.Flatten(-1),
        nn.Dropout(self.dropout),
        nn.Linear(1024, 4096),
        act_func,
        nn.Dropout(self.dropout),
        nn.Linear(4096, 4096),
        act_func,
        nn.Linear(4096, 28),
        )

  def forward(self, X):
      X = self.features(X)
      batch_size, _, _, _ = X.size()
      X = X.view(batch_size, -1)
      X = self.classifier(X)
      return X

**predicting domain**

In [ ]:
alexnet_large_kernel_domain = model_training(model= AlexNetLargeKernel(configs).to(device), 
                                root_dir=root_dir, 
                                label_type=label_type, 
                                batch_size=batch_size, 
                                max_epochs=epochs, 
                                learning_rate=lr, 
                                weight_decay=weight_decay,
                                experiment_name=experiment_name)

**predicting class**

In [ ]:
alexnet_large_kernel_category = model_training(model= AlexNetLargeKernel(configs).to(device), 
                                root_dir=root_dir, 
                                label_type=label_type, 
                                batch_size=batch_size, 
                                max_epochs=epochs, 
                                learning_rate=lr, 
                                weight_decay=weight_decay,
                                experiment_name=experiment_name)

## New Section

In [ ]:
root_dir='/content/drive/MyDrive/DL/HW2/pacs_dataset'

'''# Hyperparameters for Part I
flags.DEFINE_float('learning_rate', 1e-3, 'Learning rate.')
flags.DEFINE_float('weight_decay', 0, 'Weight decay (L2 regularization).')
flags.DEFINE_integer('batch_size', 128, 'Number of examples per batch.')
flags.DEFINE_integer('epochs', 100, 'Number of epochs for training.')
flags.DEFINE_string('experiment_name', 'exp', 'Defines experiment name.')
flags.DEFINE_enum('label_type', 'domain', ['domain', 'category'],
                  'Specifies prediction task.')
                  
LABEL_SIZE = {'domain': 4, 'category': 7}'''

train_dataset = PACSDataset(root_dir=root_dir,
                              label_type='domain',
                              is_training=True)
train_loader = DataLoader(train_dataset,
                          batch_size=128,
                          shuffle=True,
                          num_workers=4)

val_dataset = PACSDataset(root_dir=root_dir,
                          label_type='domain',
                          is_training=False)
val_loader = DataLoader(val_dataset,
                        batch_size=128,
                        shuffle=False,
                        num_workers=4)

Processing Folder: domain=photo, category=dog: 100%|██████████| 169/169 [00:01<00:00, 93.31it/s]
Processing Folder: domain=photo, category=elephant: 100%|██████████| 181/181 [00:01<00:00, 97.15it/s]
Processing Folder: domain=photo, category=giraffe: 100%|██████████| 165/165 [00:02<00:00, 72.75it/s]
Processing Folder: domain=photo, category=person: 100%|██████████| 383/383 [00:04<00:00, 91.19it/s]
Processing Folder: domain=art_painting, category=giraffe: 100%|██████████| 254/254 [00:03<00:00, 77.80it/s]
Processing Folder: domain=art_painting, category=house: 100%|██████████| 262/262 [00:02<00:00, 94.12it/s] 
Processing Folder: domain=art_painting, category=dog: 100%|██████████| 348/348 [00:03<00:00, 102.02it/s]
Processing Folder: domain=art_painting, category=person: 100%|██████████| 404/404 [00:04<00:00, 99.85it/s] 
Processing Folder: domain=art_painting, category=horse: 100%|██████████| 179/179 [00:01<00:00, 105.06it/s]
Processing Folder: domain=art_painting, category=elephant: 100%|█

In [ ]:
LABEL_SIZE = {'domain': 4, 'category': 7}
label_type = 'domain'
epochs = 100
lr=1e-3
weight_decay=0


best_model = None
best_acc = 0.0

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

experiment_name = 'experiments/{}/{}_lr_{}.wd_{}'.format(
    'exp', 'domain', 1e-3, 0)

os.makedirs(experiment_name, exist_ok=True)
writer = SummaryWriter(log_dir=experiment_name)

configs = {'num_classes': LABEL_SIZE['domain', 'category']}

############################################################################
"""After implementing all required models, you can switch from here."""
model = AlexNet(configs).to(device)
# model = AlexNetLargeKernel(configs).to(device)
# model = AlexNetAvgPooling(configs).to(device)
############################################################################

print('Model Architecture:\n%s' % model)

criterion = nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(),
                              lr=lr,
                              weight_decay=weight_decay)

try:
  for epoch in range(epochs):
    for phase in ('train', 'eval'):
      if phase == 'train':
        model.train()
        dataset = train_dataset
        data_loader = train_loader
      else:
        model.eval()
        dataset = val_dataset
        data_loader = val_loader

      running_loss = 0.0
      running_corrects = 0

      for step, (images, labels) in enumerate(data_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(phase == 'train'):
          outputs = model(images)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)

          if phase == 'train':
            loss.backward()
            optimizer.step()

            writer.add_scalar('Loss/{}'.format(phase), loss.item(),
                              epoch * len(data_loader) + step)

        running_loss += loss.item() * images.size(0)
        running_corrects += torch.sum(preds == labels.data)

      epoch_loss = running_loss / len(dataset)
      epoch_acc = running_corrects.double() / len(dataset)
      writer.add_scalar('Epoch_Loss/{}'.format(phase), epoch_loss, epoch)
      writer.add_scalar('Epoch_Accuracy/{}'.format(phase), epoch_acc, epoch)
      print('[Epoch %d] %s accuracy: %.4f, loss: %.4f' %
            (epoch + 1, phase, epoch_acc, epoch_loss))

      if phase == 'eval':
        if epoch_acc > best_acc:
          best_acc = epoch_acc
          best_model = copy.deepcopy(model.state_dict())
          torch.save(best_model, os.path.join(experiment_name, 'best_model.pt'))
except KeyboardInterrupt:
    pass

Model Architecture:
AlexNet(
  (loss): CrossEntropyLoss()
  (train_acc): Accuracy()
  (val_acc): Accuracy()
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=-1, end_dim=-1)
    (1): Dropout(p=0.2, inplace=False)
    (2): Linear(in_features=9216, out_features=409

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[Epoch 1] train accuracy: 0.5176, loss: 2.0146
[Epoch 1] eval accuracy: 0.7210, loss: 0.5808
[Epoch 2] train accuracy: 0.7534, loss: 0.5599
[Epoch 2] eval accuracy: 0.7510, loss: 0.5494
[Epoch 3] train accuracy: 0.7782, loss: 0.5006
[Epoch 3] eval accuracy: 0.7790, loss: 0.4499
[Epoch 4] train accuracy: 0.7884, loss: 0.5161
[Epoch 4] eval accuracy: 0.7905, loss: 0.5580
[Epoch 5] train accuracy: 0.8006, loss: 0.4998
[Epoch 5] eval accuracy: 0.8143, loss: 0.4538
[Epoch 6] train accuracy: 0.8122, loss: 0.4536
[Epoch 6] eval accuracy: 0.8133, loss: 0.4488
[Epoch 7] train accuracy: 0.8244, loss: 0.4251
[Epoch 7] eval accuracy: 0.8257, loss: 0.4128
[Epoch 8] train accuracy: 0.8351, loss: 0.3980
[Epoch 8] eval accuracy: 0.8247, loss: 0.3885
[Epoch 9] train accuracy: 0.8420, loss: 0.3703
[Epoch 9] eval accuracy: 0.8278, loss: 0.3837
[Epoch 10] train accuracy: 0.8553, loss: 0.3448
[Epoch 10] eval accuracy: 0.8257, loss: 0.3582
[Epoch 11] train accuracy: 0.8693, loss: 0.3106
[Epoch 11] eval accu

In [ ]:
root_dir='/content/drive/MyDrive/DL/HW2/pacs_dataset'

'''# Hyperparameters for Part I
flags.DEFINE_float('learning_rate', 1e-3, 'Learning rate.')
flags.DEFINE_float('weight_decay', 0, 'Weight decay (L2 regularization).')
flags.DEFINE_integer('batch_size', 128, 'Number of examples per batch.')
flags.DEFINE_integer('epochs', 100, 'Number of epochs for training.')
flags.DEFINE_string('experiment_name', 'exp', 'Defines experiment name.')
flags.DEFINE_enum('label_type', 'domain', ['domain', 'category'],
                  'Specifies prediction task.')
                  
LABEL_SIZE = {'domain': 4, 'category': 7}'''

train_dataset_cat = PACSDataset(root_dir=root_dir,
                              label_type='category',
                              is_training=True)
train_loader_cat = DataLoader(train_dataset,
                          batch_size=128,
                          shuffle=True,
                          num_workers=4)

val_dataset_cat = PACSDataset(root_dir=root_dir,
                          label_type='category',
                          is_training=False)
val_loader_cat = DataLoader(val_dataset,
                        batch_size=128,
                        shuffle=False,
                        num_workers=4)

Processing Folder: domain=photo, category=dog: 100%|██████████| 169/169 [00:01<00:00, 92.45it/s] 
Processing Folder: domain=photo, category=elephant: 100%|██████████| 181/181 [00:01<00:00, 96.99it/s] 
Processing Folder: domain=photo, category=giraffe: 100%|██████████| 165/165 [00:01<00:00, 94.62it/s] 
Processing Folder: domain=photo, category=person: 100%|██████████| 383/383 [00:04<00:00, 94.22it/s] 
Processing Folder: domain=art_painting, category=giraffe: 100%|██████████| 254/254 [00:02<00:00, 96.75it/s] 
Processing Folder: domain=art_painting, category=house: 100%|██████████| 262/262 [00:02<00:00, 94.50it/s] 
Processing Folder: domain=art_painting, category=dog: 100%|██████████| 348/348 [00:03<00:00, 97.39it/s] 
Processing Folder: domain=art_painting, category=person: 100%|██████████| 404/404 [00:04<00:00, 96.28it/s] 
Processing Folder: domain=art_painting, category=horse: 100%|██████████| 179/179 [00:01<00:00, 92.94it/s] 
Processing Folder: domain=art_painting, category=elephant: 1

In [ ]:
LABEL_SIZE = {'domain': 4, 'category': 7}
label_type = 'category'
epochs = 50
lr=1e-3
weight_decay=0


best_model_cat = None
best_acc = 0.0

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

experiment_name = 'experiments/{}/{}_lr_{}.wd_{}'.format(
    'exp', 'category', 1e-3, 0)

os.makedirs(experiment_name, exist_ok=True)
writer = SummaryWriter(log_dir=experiment_name)

configs = {'num_classes': LABEL_SIZE['category']}

############################################################################
"""After implementing all required models, you can switch from here."""
model = AlexNet(configs).to(device)
# model = AlexNetLargeKernel(configs).to(device)
# model = AlexNetAvgPooling(configs).to(device)
############################################################################

print('Model Architecture:\n%s' % model)

criterion = nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(),
                              lr=lr,
                              weight_decay=weight_decay)

try:
  for epoch in range(epochs):
    for phase in ('train', 'eval'):
      if phase == 'train':
        model.train()
        dataset = train_dataset
        data_loader = train_loader
      else:
        model.eval()
        dataset = val_dataset
        data_loader = val_loader

      running_loss = 0.0
      running_corrects = 0

      for step, (images, labels) in enumerate(data_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(phase == 'train'):
          outputs = model(images)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)

          if phase == 'train':
            loss.backward()
            optimizer.step()

            writer.add_scalar('Loss/{}'.format(phase), loss.item(),
                              epoch * len(data_loader) + step)

        running_loss += loss.item() * images.size(0)
        running_corrects += torch.sum(preds == labels.data)

      epoch_loss = running_loss / len(dataset)
      epoch_acc = running_corrects.double() / len(dataset)
      writer.add_scalar('Epoch_Loss/{}'.format(phase), epoch_loss, epoch)
      writer.add_scalar('Epoch_Accuracy/{}'.format(phase), epoch_acc, epoch)
      print('[Epoch %d] %s accuracy: %.4f, loss: %.4f' %
            (epoch + 1, phase, epoch_acc, epoch_loss))

      if phase == 'eval':
        if epoch_acc > best_acc:
          best_acc = epoch_acc
          best_model_cat = copy.deepcopy(model.state_dict())
          torch.save(best_model_cat, os.path.join(experiment_name, 'best_model_cat.pt'))
except KeyboardInterrupt:
    pass

Model Architecture:
AlexNet(
  (loss): CrossEntropyLoss()
  (train_acc): Accuracy()
  (val_acc): Accuracy()
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=-1, end_dim=-1)
    (1): Dropout(p=0.2, inplace=False)
    (2): Linear(in_features=9216, out_features=409

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[Epoch 1] train accuracy: 0.5474, loss: 1.9022
[Epoch 1] eval accuracy: 0.7718, loss: 0.5308
[Epoch 2] train accuracy: 0.7656, loss: 0.5426
[Epoch 2] eval accuracy: 0.7573, loss: 0.5705
[Epoch 3] train accuracy: 0.7830, loss: 0.5213
[Epoch 3] eval accuracy: 0.7936, loss: 0.4759
[Epoch 4] train accuracy: 0.7978, loss: 0.4913
[Epoch 4] eval accuracy: 0.7967, loss: 0.4731
[Epoch 5] train accuracy: 0.7976, loss: 0.4827
[Epoch 5] eval accuracy: 0.8154, loss: 0.4403
[Epoch 6] train accuracy: 0.8219, loss: 0.4254
[Epoch 6] eval accuracy: 0.8444, loss: 0.3788
[Epoch 7] train accuracy: 0.8355, loss: 0.4008
[Epoch 7] eval accuracy: 0.8278, loss: 0.4295
[Epoch 8] train accuracy: 0.8347, loss: 0.4171
[Epoch 8] eval accuracy: 0.8288, loss: 0.3881
[Epoch 9] train accuracy: 0.8513, loss: 0.3653
[Epoch 9] eval accuracy: 0.8496, loss: 0.3543
[Epoch 10] train accuracy: 0.8294, loss: 0.4457
[Epoch 10] eval accuracy: 0.8216, loss: 0.4258
[Epoch 11] train accuracy: 0.8450, loss: 0.3951
[Epoch 11] eval accu

# **3. Visualizing Learned Filter**

In [ ]:
def analyze_model_kernels(module, ckpt_path):
    loaded_model = module.load_from_checkpoint(ckpt_path)
    module_name = os.path.split(dirname(ckpt_path))[-1]
    layers = loaded_model.features
    for layer_num, layer in enumerate(layers):
        if isinstance(layer, nn.Conv2d):
            kernel_name = str(layer).split('(', 1)[0]
            kernel_weight = loaded_model.features[layer_num].weight.data
            visualize_kernels(kernel_name, kernel_weight, layer_num, saving_prefix=module_name)

In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import dirname


def visualize_kernels(kernel_name,
                      kernel_weight,
                      max_in_channels=12,
                      max_out_channels=12,
                      saving_prefix='kernel'):
  """A helper function to visualize the learned convolutional kernels.
  
  Args:
    kernel_name: str, the name of the kernel being visualized. It will be used
        as the filename in the saved figures.
    kernel_weight: torch.Tensor or np.ndarray, the weights of convolutional
        kernel. The shape should be
        [out_channels, in_channels, kernel_height, kernel_width].
    max_in_channels: int, optional, the max in_channels in the visualization.
    max_out_channels: int, optional, the max out_channels in the visualization.
    saving_prefix: str, optional, the directory for saving the visualization.
  """
  print('Visualize the learned filter of `%s`' % kernel_name)
  if isinstance(kernel_weight, torch.Tensor):
    kernel_weight = kernel_weight.cpu().numpy()

  kernel_shape = list(kernel_weight.shape)

  nrows = min(max_in_channels, kernel_shape[1])
  ncols = min(max_out_channels, kernel_shape[0])

  fig, axes = plt.subplots(nrows, ncols, figsize=(ncols, nrows))

  for r in range(nrows):
    for c in range(ncols):
      kernel = kernel_weight[c, r, :, :]
      vmin, vmax = kernel.min(), kernel.max()
      normalized_kernel = (kernel - vmin) / (vmax - vmin)
      sns.heatmap(normalized_kernel,
                  cbar=False,
                  square=True,
                  xticklabels=False,
                  yticklabels=False,
                  ax=axes[r, c])

  plt.xlabel('First %d In-Channels' % nrows)
  plt.ylabel('First %d Out-Channels' % ncols)

  plt.tight_layout()
  plt.savefig(os.path.join(saving_prefix, kernel_name.lower() + '.png'))
  return
